In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install tensorflow==2.15.0

In [ ]:
import os
import subprocess
import nibabel as nib
import tensorflow as tf
from skimage import exposure
from math import ceil
import matplotlib.pyplot as plt
import random
from torch import nn
from torch.autograd import Variable
import torch
import torch.nn.functional as F

In [ ]:
import os
import subprocess
import nibabel as nib
import tensorflow as tf
from skimage import exposure
from math import ceil
import matplotlib.pyplot as plt
import random

class DataLoader:
    def __init__(self, data_path, split_ratio, batch_size=None):
        self.data_path = data_path
        self.split_ratio = split_ratio
        self.all_subjects = None
        self.subjects_lists = []
        self.labels = {'train': 0, 'test': 1, 'validation': 2}
        self.size = [0, 0, 0]
        self.batch_size = batch_size
        self.slices_number = None

    def list_subjects(self):
        subjects = os.listdir(self.data_path)
        subjects = [item for item in subjects if item.startswith('sub')]
        subjects.reverse()
        self.all_subjects = subjects

    def get_nifti_path(self, subject, number_of_motion='1'):
        ref_path_stand = f'{self.data_path}/{subject}/anat/{subject}_acq-standard_T1w.nii'
#         select_path_stand = subprocess.run(['ls', ref_path_stand], capture_output=True, text=True).stdout.replace("\n", "")

        ref_path_motion = f'{self.data_path}/{subject}/anat/{subject}_acq-headmotion{number_of_motion}_T1w.nii'
#         select_path_motion = subprocess.run(['ls', ref_path_motion], capture_output=True, text=True).stdout.replace("\n", "")

        return [ref_path_stand , ref_path_motion ]

    def get_paired_volumes(self, path,remove_black=True):
        if os.path.exists(path[0]) and os.path.exists(path[1]):
            free_data = nib.load(path[0]).get_fdata()
            
            motion_data = nib.load(path[1]).get_fdata()
            if remove_black:
                free_data = exposure.rescale_intensity(free_data[37:-37], out_range=(-1.0, 1.0))
                motion_data = exposure.rescale_intensity(motion_data[37:-37], out_range=(-1.0, 1.0))
            else:
                free_data = exposure.rescale_intensity(free_data, out_range=(-1.0, 1.0))
                motion_data = exposure.rescale_intensity(motion_data, out_range=(-1.0, 1.0))
            return tf.convert_to_tensor(free_data), tf.convert_to_tensor(motion_data)
        else:
            return None, None

    def split_data(self):
        self.list_subjects()
        if ceil(sum(self.split_ratio)) == 1 and len(self.split_ratio) <= 3:
            self.split_ratio.insert(0, 0)
            cumulative_sum = [sum(self.split_ratio[:i + 1]) for i in range(len(self.split_ratio))]
            number_of_subjects = len(self.all_subjects)

            for i in range(1, len(self.split_ratio)):
                self.subjects_lists.append(
                    self.all_subjects[int(round(cumulative_sum[i - 1] * number_of_subjects)):int(
                        round(cumulative_sum[i] * number_of_subjects))])

                self.size[i - 1] = len(self.subjects_lists[i - 1])  * 2 * 190

                if i - 1 == 0:
                    self.size[i - 1] -= 8  * 2 * 190
        else:
            print("The Summation of ratios is not equal to 1")
       
    def generator(self, mode,remove_black=True):
        subjects = self.subjects_lists[self.labels[mode]]

        def data_gen():
            for subject in subjects:
                for i in range(2):
                    pathes = self.get_nifti_path(subject, str(i + 1))
                    free, motion = self.get_paired_volumes(pathes,remove_black)
                    if motion is not None:
                        self.slices_number = motion.shape[0]

                        for slice_id in range(0, self.slices_number):
                            start_idx = slice_id + 1
                            end_idx = (slice_id + 1) + 1
                            if (end_idx < self.slices_number-1):
                                free_slice = free[start_idx:end_idx]
                                free_slice = tf.transpose(free_slice, perm=[1, 2, 0])
                                
                                motion_slice = motion[start_idx:end_idx]
                                motion_slice = tf.transpose(motion_slice, perm=[1, 2, 0])
                                
                                motion_before_slice = motion[start_idx-1:end_idx-1]
                                motion_before_slice = tf.transpose(motion_before_slice, perm=[1, 2, 0])
                                
                                motion_after_slice = motion[start_idx+1:end_idx+1]
                                motion_after_slice = tf.transpose(motion_after_slice, perm=[1, 2, 0])

                                yield (
                                (motion_before_slice, motion_slice, motion_after_slice),
                                free_slice
                                )

        input_signature = (
            (tf.TensorSpec(shape=(256, 256, 1), dtype=tf.float32),
             tf.TensorSpec(shape=(256, 256, 1), dtype=tf.float32),
             tf.TensorSpec(shape=(256, 256, 1), dtype=tf.float32)),
            tf.TensorSpec(shape=(256, 256, 1), dtype=tf.float32)
        )

        dataset = tf.data.Dataset.from_generator(data_gen, output_signature=input_signature)
        dataset = dataset.batch(self.batch_size)

        return dataset

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint, LearningRateScheduler
import math
import pandas as pd
from tensorflow.keras.models import model_from_json

# Constants
TRAIN = 1  # True False
TEST = 0  # True False
NB_EPOCH = 50
LEARNING_RATE = 0.001  # 0.001 (default)
HEIGHT, WIDTH = 256, 256
PREDICTION_PATH = '/kaggle/working/Prediction'
WEIGHTS_PATH = '/kaggle/working/Weights'

print('Reading Data ... ')
data_path = "/kaggle/input/mmmai-regist-data/MR-ART-Regist"
split_ratio = [0.7, 0.2, 0.1]
# split_ratio = [0.01, 0.98, 0.01]
batch_size = 5

data_loader = DataLoader(data_path, split_ratio, batch_size)
data_loader.split_data()

train_dataset = data_loader.generator('train')
test_dataset = data_loader.generator('test',False)
validation_dataset = data_loader.generator('validation')

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
from tensorflow.keras import backend as K
from tensorflow.keras.losses import MeanSquaredError
import numpy as np

IMAGE_ORDERING = 'channels_last'
MERGE_AXIS = -1

def cbam_block(cbam_feature, ratio=8):
    cbam_feature = channel_attention(cbam_feature, ratio)
    cbam_feature = spatial_attention(cbam_feature)
    return cbam_feature

def channel_attention(input_feature, ratio=8):
    channel_axis = -1
    channel = input_feature.shape[channel_axis]

    shared_layer_one = Dense(channel//ratio, activation='relu', kernel_initializer='he_normal', use_bias=True, bias_initializer='zeros')
    shared_layer_two = Dense(channel, kernel_initializer='he_normal', use_bias=True, bias_initializer='zeros')

    avg_pool = GlobalAveragePooling2D()(input_feature)
    avg_pool = Reshape((1,1,channel))(avg_pool)
    avg_pool = shared_layer_one(avg_pool)
    avg_pool = shared_layer_two(avg_pool)

    max_pool = GlobalMaxPooling2D()(input_feature)
    max_pool = Reshape((1,1,channel))(max_pool)
    max_pool = shared_layer_one(max_pool)
    max_pool = shared_layer_two(max_pool)

    cbam_feature = Add()([avg_pool,max_pool])
    cbam_feature = Activation('sigmoid')(cbam_feature)
    
    cbam_feature = 1-cbam_feature
    
    return multiply([input_feature, cbam_feature])
def spatial_attention(input_feature):
    kernel_size = 7

    avg_pool = Lambda(lambda x: K.mean(x, axis=3, keepdims=True))(input_feature)
    max_pool = Lambda(lambda x: K.max(x, axis=3, keepdims=True))(input_feature)
    concat = Concatenate(axis=3)([avg_pool, max_pool])
    cbam_feature = Conv2D(filters = 1, kernel_size=kernel_size, strides=1, padding='same', activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(concat)

    return multiply([input_feature, cbam_feature])
def expand_moments_dim(moment):
    return tf.reshape(moment, [-1, 1, 1, tf.shape(moment)[-1]])

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
def ms_ssim_score(y_true, y_pred):
    score = tf.reduce_mean(tf.image.ssim_multiscale(y_true, y_pred, 2.0))
    return score
def ms_ssim_loss(y_true, y_pred):
    loss_ssim = 1.0 - tf.reduce_mean(tf.image.ssim_multiscale(y_true, y_pred, 2.0))
    return loss_ssim
def ssim_score(y_true, y_pred):
    score = tf.reduce_mean(tf.image.ssim(y_true, y_pred, 2.0,filter_size=5,
                          filter_sigma=1.5))
    return score

def ssim_loss(y_true, y_pred):
    loss_ssim = 1.0 - tf.reduce_mean(tf.image.ssim(y_true, y_pred, 2.0,filter_size=5,
                          filter_sigma=1.5))
    return loss_ssim
def l1_loss(y_true, y_pred):
    return tf.reduce_mean(tf.abs(y_true - y_pred))
def l2_loss(y_true, y_pred):
    """
    Computes the L2 loss between the ground truth and predicted tensors.

    Parameters:
        y_true (tf.Tensor): Ground truth tensor.
        y_pred (tf.Tensor): Predicted tensor.

    Returns:
        tf.Tensor: Normalized L2 loss.

    This function calculates the mean squared error (MSE) between the ground truth
    and predicted tensors. It then reduces the MSE along the spatial dimensions,
    typically representing the height and width of the tensors, resulting in a
    tensor of shape (batch_size,), where each element represents the mean MSE
    for a single sample in the batch.

    The loss is then normalized using L2 normalization to ensure that it falls
    within the range of 0 to 1. Finally, the mean of the normalized loss across
    the batch is computed and returned.
    """
    mse = tf.keras.losses.mean_squared_error(y_true, y_pred)

    # Reduce on spatial information
    batch_mse = tf.reduce_mean(mse, axis=(1, 2))

    # Normalize the loss function to be between 0 and 1
    normalized_loss = tf.nn.l2_normalize(batch_mse, axis=-1)
    
    # Compute the mean of the normalized loss across the batch
    normalized_reduced_loss = tf.reduce_mean(batch_mse)

    return normalized_reduced_loss

def spatial_fft_loss(y_true, y_pred):
    """
    Custom loss function for spatial loss with FFT features.

    Args:
        y_true: Ground truth image(s).
        y_pred: Predicted image(s).

    Returns:
        Normalized reduced spatial loss.

    This function defines a custom loss for training neural networks. It applies a Fourier Transform
    to the true and predicted images, extracts the real and imaginary parts of the transformed
    features, and calculates the mean squared error between them. The loss is then normalized and
    reduced to a single scalar value.

    """
    # Apply Fourier Transform to the true and predicted images
    true_fft = tf.signal.fft2d(tf.cast(y_true, dtype=tf.complex64))
    pred_fft = tf.signal.fft2d(tf.cast(y_pred, dtype=tf.complex64))

    # Extract Real & Imaginary parts
    true_fft_real = tf.math.real(true_fft)
    true_fft_imag = tf.math.imag(true_fft)
    pred_fft_real = tf.math.real(pred_fft)
    pred_fft_imag = tf.math.imag(pred_fft)

    # Crop center rectangles for real and imag
    true_fft_real_cropped = crop_center_rectangle_mask(true_fft_real)
    true_fft_imag_cropped = crop_center_rectangle_mask(true_fft_imag)
    pred_fft_real_cropped = crop_center_rectangle_mask(pred_fft_real)
    pred_fft_imag_cropped = crop_center_rectangle_mask(pred_fft_imag)

    # Calculate L2 loss
    mse_real = tf.keras.losses.mean_squared_error(true_fft_real_cropped, pred_fft_real_cropped)
    mse_imag = tf.keras.losses.mean_squared_error(true_fft_imag_cropped, pred_fft_imag_cropped)

    # Total L2 loss
    total_loss = 0.5 * (mse_real + mse_imag)
    
    # Reduce on spatial information
    batch_loss = tf.reduce_mean(total_loss, axis=(1, 2))
    
    # Normalize the loss function to be between 0 and 1
    normalized_loss = tf.nn.l2_normalize(batch_loss, axis=-1)

    normalized_reduced_loss = tf.reduce_mean(normalized_loss)

    return normalized_reduced_loss

import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model

def init_vgg16_model():
    """
    Initialize a pre-trained VGG16 model for feature extraction.

    Args:
        perceptual_layer_name: Name of the layer to extract features from.

    Returns:
        Pre-trained VGG16 model with specified layer for feature extraction.

    This function loads a pre-trained VGG16 model with ImageNet weights and removes the top
    classification layers. It then extracts the specified layer for feature extraction and
    freezes the model's layers to prevent further training.

    """
    # Load pre-trained VGG16 model without the top classification layers
    vgg_model = VGG16(include_top=False, weights='imagenet', input_shape=(256, 256, 3))

    # Extract the specified layer from the VGG16 model
    perceptual_model_conv1 = Model(inputs=vgg_model.input, outputs=vgg_model.get_layer('block1_conv1').output)
    perceptual_model_conv2 = Model(inputs=vgg_model.input, outputs=vgg_model.get_layer('block2_conv1').output)
    perceptual_model_conv3 = Model(inputs=vgg_model.input, outputs=vgg_model.get_layer('block3_conv1').output)

    # Freeze the layers in the perceptual model so they are not trained further
    for perceptual_model in [perceptual_model_conv1,perceptual_model_conv2,perceptual_model_conv3]:
        for layer in perceptual_model.layers:
            layer.trainable = False
        
    print("VGG16 Model Initialized")
    return perceptual_model_conv1, perceptual_model_conv2, perceptual_model_conv3

# Initialize VGG16 model for feature extraction
perceptual_models = init_vgg16_model()

def perceptual_loss(y_true, y_pred):
    """
    Custom loss function for perceptual loss.

    Args:
        y_true: Ground truth image(s).
        y_pred: Predicted image(s).

    Returns:
        Normalized reduced perceptual loss.

    This function defines a custom loss for training neural networks. It converts single-channel
    images to RGB, preprocesses them for VGG16, and extracts features from a specified layer
    using a pre-trained VGG16 model. It then calculates the mean squared error between the features
    of the true and predicted images. The loss is normalized and reduced to a single scalar value.

    """
    # Extract perceptual models
    perceptual_model_conv1, perceptual_model_conv2, perceptual_model_conv3 = perceptual_models

    # Convert single-channel images to RGB
    y_true_rgb = tf.repeat(y_true, 3, axis=-1)
    y_pred_rgb = tf.repeat(y_pred, 3, axis=-1)

    # Preprocess images for VGG16
    y_true_processed = tf.keras.applications.vgg16.preprocess_input(y_true_rgb)
    y_pred_processed = tf.keras.applications.vgg16.preprocess_input(y_pred_rgb)

    # Extract features from specified layer for true and predicted images
    features_true_conv1 = perceptual_model_conv1(y_true_processed)
    features_pred_conv1 = perceptual_model_conv1(y_pred_processed)
   
    # Extract features from specified layer for true and predicted images
    features_true_conv2 = perceptual_model_conv2(y_true_processed)
    features_pred_conv2 = perceptual_model_conv2(y_pred_processed)
  
    # Extract features from specified layer for true and predicted images
    features_true_conv3 = perceptual_model_conv3(y_true_processed)
    features_pred_conv3 = perceptual_model_conv3(y_pred_processed)
    
    # Calculate L2 loss
    mse_conv1 = tf.reduce_mean(tf.keras.losses.mse(features_true_conv1, features_pred_conv1))
    mse_conv2 = tf.reduce_mean(tf.keras.losses.mse(features_true_conv2, features_pred_conv2))
    mse_conv3 = tf.reduce_mean(tf.keras.losses.mse(features_true_conv3, features_pred_conv3))
    
    total_loss = 0.65*mse_conv1 + 0.3*mse_conv2 + 0.05*mse_conv3

    return total_loss


def psnr(y_true, y_pred):
    return tf.reduce_mean(tf.image.psnr(y_true, y_pred, max_val=2.0))  # Adjust max_val for data normalized between -1 and 1

def total_loss(y_true, y_pred):
    perceptual = perceptual_loss(y_true, y_pred)
    ssim = ssim_loss(y_true, y_pred)
    
    scaled_perceptual = (perceptual*0.05807468295097351)
    adjusted_perceptual = (scaled_perceptual+0.009354699403047562)
    
    total = (ssim+adjusted_perceptual)/2
    return total


In [ ]:
from tensorflow.keras import *
import math
import pandas as pd
import tensorflow as tf
from tensorflow.keras.callbacks import CSVLogger, LearningRateScheduler, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import model_from_json
LEARNING_RATE = 0.001
def UNet(img_input,norm_list):
    k1 = 32
    k2 = 64
    k3 = 128
    k4 = 256
    # Block 1 in Contracting Path
    conv1 = Conv2D(k1, (3, 3), data_format=IMAGE_ORDERING,padding='same', dilation_rate=1)(img_input)
    conv1 = BatchNormalization()(conv1)
    conv1 = Activation(tf.nn.leaky_relu)(conv1)
    #conv1 = Dropout(0.2)(conv1)
    conv1 = Conv2D(k1, (3, 3), data_format=IMAGE_ORDERING, padding='same', dilation_rate=1)(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = Activation(tf.nn.leaky_relu)(conv1)

    conv1 = cbam_block(conv1)    # Convolutional Block Attention Module(CBAM) block

    o = AveragePooling2D((2, 2), strides=(2, 2))(conv1)

    # Block 2 in Contracting Path
    conv2 = Conv2D(k2, (3, 3), data_format=IMAGE_ORDERING, padding='same', dilation_rate=1)(o)
    conv2 = BatchNormalization()(conv2)
    conv2 = Activation(tf.nn.leaky_relu)(conv2)
    conv2 = Dropout(0.2)(conv2)
    conv2 = Conv2D(k2, (3, 3), data_format=IMAGE_ORDERING, padding='same', dilation_rate=1)(conv2)
    conv2 = BatchNormalization()(conv2)
    conv2 = Activation(tf.nn.leaky_relu)(conv2)

    conv2 = cbam_block(conv2)    # Convolutional Block Attention Module(CBAM) block

    o = AveragePooling2D((2, 2), strides=(2, 2))(conv2)

    # Block 3 in Contracting Path
    conv3 = Conv2D(k3, (3, 3), data_format=IMAGE_ORDERING, padding='same', dilation_rate=1)(o)
    conv3 = BatchNormalization()(conv3)
    conv3 = Activation(tf.nn.leaky_relu)(conv3)
    #conv3 = Dropout(0.2)(conv3)
    conv3 = Conv2D(k3, (3, 3), data_format=IMAGE_ORDERING, padding='same', dilation_rate=1)(conv3)
    conv3 = BatchNormalization()(conv3)
    conv3 = Activation(tf.nn.leaky_relu)(conv3)

    conv3 = cbam_block(conv3)    # Convolutional Block Attention Module(CBAM) block

    o = AveragePooling2D((2, 2), strides=(2, 2))(conv3)

     # Transition layer between contracting and expansive paths:
    conv4 = Conv2D(k4, (3, 3), data_format=IMAGE_ORDERING, padding='same', dilation_rate=1)(o)
    conv4 = BatchNormalization()(conv4)
    conv4 = Activation(tf.nn.leaky_relu)(conv4)
    #conv4 = Dropout(0.2)(conv4)
    conv4 = Conv2D(k4, (3, 3), data_format=IMAGE_ORDERING, padding='same', dilation_rate=1)(conv4)
    conv4 = BatchNormalization()(conv4)
    conv4 =Activation(tf.nn.leaky_relu)(conv4)

    conv4 = cbam_block(conv4)    # Convolutional Block Attention Module(CBAM) block
    res1 = Conv2D(k4, (3, 3),padding='same',activation='relu')(conv4)
    AdaptiveInstanceNorm()(res1,norm_list[:,:,:,:k4], norm_list[:,:,:,k4:2*k4])
    res1 = Conv2D(k4, (3, 3),padding='same')(res1)
# AdaptiveInstanceNorm()(res1,norm_list[:,:,:,:k4], norm_list[:,:,:,k4:2*k4])
    res1 += conv4
    
    res2 = Conv2D(k4, (3, 3),padding='same',activation='relu')(res1)
    AdaptiveInstanceNorm()(res2,norm_list[:,:,:,2*k4:3*k4], norm_list[:,:,:,3*k4:4*k4])
    res2 = Conv2D(k4, (3, 3),padding='same')(res2)
# 	AdaptiveInstanceNorm()(res2,norm_list[:,:,:,2*k4:3*k4], norm_list[:,:,:,3*k4:4*k4])
    res2 += res1
    
    res3 = Conv2D(k4, (3, 3),padding='same',activation='relu')(res2)
    AdaptiveInstanceNorm()(res3,norm_list[:,:,:,4*k4:5*k4], norm_list[:,:,:,4*k4:5*k4])
    res3 = Conv2D(k4, (3, 3),padding='same')(res3)
# 	AdaptiveInstanceNorm()(res3,norm_list[:,:,:,4*k4:5*k4], norm_list[:,:,:,4*k4:5*k4])
    res3 += res2
    
    res4 = Conv2D(k4, (3, 3),padding='same',activation='relu')(res3)
    AdaptiveInstanceNorm()(res4,norm_list[:,:,:,5*k4:6*k4], norm_list[:,:,:,5*k4:6*k4])
    res4 = Conv2D(k4, (3, 3),padding='same')(res4)
# 	AdaptiveInstanceNorm()(res4,norm_list[:,:,:,5*k4:6*k4], norm_list[:,:,:,5*k4:6*k4])
    res4 += res3
    conv4 = res4
    
    
	# Block 1 in Expansive Path
    up1 = UpSampling2D((2, 2), data_format=IMAGE_ORDERING)(conv4)
    up1 = concatenate([up1, conv3], axis=MERGE_AXIS)
    deconv1 =  Conv2D(k3, (3, 3), data_format=IMAGE_ORDERING, padding='same', dilation_rate=1)(up1)
    deconv1 = BatchNormalization()(deconv1)
# 	deconv1 = AdaptiveInstanceNorm()(deconv1,norm_list[:,:,:,:128], norm_list[:,:,:,128:256])
    deconv1 = Activation(tf.nn.leaky_relu)(deconv1)
	#deconv1 = Dropout(0.2)(deconv1)
    deconv1 =  Conv2D(k3, (3, 3), data_format=IMAGE_ORDERING, padding='same', dilation_rate=1)(deconv1)
    deconv1 = BatchNormalization()(deconv1)
# 	deconv1 = AdaptiveInstanceNorm()(deconv1,norm_list[:,:,:,:128], norm_list[:,:,:,128:256])
    deconv1 = Activation(tf.nn.leaky_relu)(deconv1)

    deconv1 = cbam_block(deconv1)    # Convolutional Block Attention Module(CBAM) block

	# Block 2 in Expansive Path
    up2 = UpSampling2D((2, 2), data_format=IMAGE_ORDERING)(deconv1)
    up2 = concatenate([up2, conv2], axis=MERGE_AXIS)
    deconv2 = Conv2D(k2, (3, 3), data_format=IMAGE_ORDERING, padding='same', dilation_rate=1)(up2)
    deconv2 = BatchNormalization()(deconv2)
# 	deconv2 = AdaptiveInstanceNorm()(deconv2,norm_list[:,:,:,256:320], norm_list[:,:,:,320:384])
    deconv2 = Activation(tf.nn.leaky_relu)(deconv2)
	#deconv2 = Dropout(0.2)(deconv2)
    deconv2 = Conv2D(k2, (3, 3), data_format=IMAGE_ORDERING, padding='same', dilation_rate=1)(deconv2)
    deconv2 = BatchNormalization()(deconv2)
# 	deconv2 = AdaptiveInstanceNorm()(deconv2,norm_list[:,:,:,256:320], norm_list[:,:,:,320:384])
    deconv2 = Activation(tf.nn.leaky_relu)(deconv2)

    deconv2 = cbam_block(deconv2)    # Convolutional Block Attention Module(CBAM) block

	# Block 3 in Expansive Path
    up3 = UpSampling2D((2, 2), data_format=IMAGE_ORDERING)(deconv2)
    up3 = concatenate([up3, conv1], axis=MERGE_AXIS)
    deconv3 = Conv2D(k1, (3, 3), data_format=IMAGE_ORDERING, padding='same', dilation_rate=1)(up3)
    deconv3 = BatchNormalization()(deconv3)
# deconv3 = AdaptiveInstanceNorm()(deconv3,norm_list[:,:,:,384:416], norm_list[:,:,:,416:448])
    deconv3 = Activation(tf.nn.leaky_relu)(deconv3)
    #deconv3 = Dropout(0.2)(deconv3)
    deconv3 = Conv2D(k1, (3, 3), data_format=IMAGE_ORDERING, padding='same', dilation_rate=1)(deconv3)
    deconv3 = BatchNormalization()(deconv3)
#  deconv3 = AdaptiveInstanceNorm()(deconv3,norm_list[:,:,:,384:416], norm_list[:,:,:,416:448])
    deconv3 = Activation(tf.nn.leaky_relu)(deconv3)

    deconv3 = cbam_block(deconv3)    # Convolutional Block Attention Module(CBAM) block

    output = Conv2D(1, (3, 3), data_format=IMAGE_ORDERING, padding='same')(deconv3)
# 	output = Activation('sigmoid')(output)
    output = Activation('tanh')(output)
    return output
class AdaptiveInstanceNorm(Layer):
    def __init__(self, epsilon=1e-5, **kwargs):
        super(AdaptiveInstanceNorm, self).__init__(**kwargs)
        self.epsilon = epsilon

    def call(self, content, gamma, beta):
        c_mean, c_var = tf.nn.moments(content, axes=[1, 2], keepdims=True)
        c_std = tf.sqrt(c_var + self.epsilon)
        normalized = (content - c_mean) / c_std
        gamma = expand_moments_dim(gamma)
        beta = expand_moments_dim(beta)
        return multiply([gamma, normalized]) + beta

    def get_config(self):
        config = super(AdaptiveInstanceNorm, self).get_config()
        config.update({"epsilon": self.epsilon})
    
def Style_Encoder(style_dim, img_input):
    initializer = tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.02)
    k1, k2, k3 = 32, 64, 128
    conv_1 = Conv2D(k1, (7, 7), padding="same", strides=1, activation='relu', kernel_initializer=initializer)(img_input)
    conv_2 = Conv2D(k2, (4, 4), padding="same", strides=2, activation='relu', kernel_initializer=initializer)(conv_1)
    conv_3 = Conv2D(k3, (4, 4), padding="same", strides=2, activation='relu', kernel_initializer=initializer)(conv_2)
    conv_4 = Conv2D(k3, (4, 4), padding="same", strides=2, activation='relu', kernel_initializer=initializer)(conv_3)
    conv_5 = Conv2D(k3, (4, 4), padding="same", strides=2, activation='relu', kernel_initializer=initializer)(conv_4)
    avg = GlobalAveragePooling2D(keepdims=True)(conv_5)
    conv_6 = Conv2D(style_dim, (1, 1), padding="valid", strides=1, kernel_initializer=initializer)(avg)
    return conv_6

def MLP(input_block, output_dim, dim):
    layer1 = Dense(dim, activation='relu', use_bias=True)(input_block)
    layer2 = Dense(dim, activation='relu', use_bias=True)(layer1)
    layer3 = Dense(output_dim, activation='relu', use_bias=True)(layer2)
    return layer3
def Feature_extractor(img_input_1, img_input_2,img_input_3):
#     assert input_height % 32 == 0
#     assert input_width % 32 == 0

# #   UNET
#     img_input_1 = Input(shape=(input_height, input_width, 1))
#     img_input_2 = Input(shape=(input_height, input_width, 1))
#     img_input_3 = Input(shape=(input_height, input_width, 1))
# 	kk = 32
    kk = 64
    conv1 = Conv2D(kk, (3, 3), data_format=IMAGE_ORDERING,padding='same', dilation_rate=1)(img_input_1) # dilation_rate=6
    conv1 = BatchNormalization()(conv1)
    conv1 = Activation('relu')(conv1)
    conv2 = Conv2D(kk, (3, 3), data_format=IMAGE_ORDERING,padding='same', dilation_rate=1)(img_input_2) # dilation_rate=6
    conv2 = BatchNormalization()(conv2)
    conv2 = Activation('relu')(conv2)
    conv3 = Conv2D(kk, (3, 3), data_format=IMAGE_ORDERING,padding='same', dilation_rate=1)(img_input_3) # dilation_rate=6
    conv3 = BatchNormalization()(conv3)
    conv3 = Activation('relu')(conv3)

    input_concat = concatenate([conv1, conv2, conv3], axis=MERGE_AXIS)  #conv4
    return input_concat



In [ ]:
resume = True

In [ ]:
import csv
import os
from tensorflow.keras.callbacks import ModelCheckpoint
import h5py
from tensorflow.keras.models import load_model
# epochs = 10
epoch_results_path = "/kaggle/working/epoch_results_file.csv"
step_results_path = "/kaggle/working/step_results_file.csv"
model_checkpoint_path = "/kaggle/working/model_checkpoint_{epoch:02d}.h5"
# Define callbacks to save the model after each epoch
checkpoint_callback = ModelCheckpoint(
    filepath=model_checkpoint_path,
    monitor='val_loss',  # You can change the monitor value based on your requirement
    save_best_only=True,
    save_weights_only=False,
    mode='min',  # You can change the mode value based on your requirement
    verbose=1
)
if resume:
    data_of_model = os.listdir('/kaggle/input/modelstacked-unet-with-style')
    data_of_model.sort()
    curr_epoch = int((len(data_of_model)-2)/2)
    full_unet = load_model(f'/kaggle/input/modelstacked-unet-with-style/full_unet_epoch_{curr_epoch}.h5')
    full_style = load_model(f'/kaggle/input/modelstacked-unet-with-style/full_style_epoch_{curr_epoch}.h5')
# feature_input = Input(shape=(256, 256, 192))
else:
    norm_list = Input(shape=(1, 1, 1536))  # Adjust the shape as necessary
    img_free = Input(shape=(256, 256, 1))
    img_input1 = Input(shape=(256, 256, 1))
    img_input2 = Input(shape=(256, 256, 1))
    img_input3 = Input(shape=(256, 256, 1))
# Create the U-Net model
    Feature_output = Feature_extractor(img_input1,img_input2,img_input3)
    Feature_model = Model(inputs=[img_input1,img_input2,img_input3], outputs=Feature_output)

    Unet_output = UNet(Feature_output, norm_list)
# Unet_model = Model(inputs=[Feature_output, norm_list], outputs=Unet_output)

    input_2 = concatenate([Feature_output, Unet_output], axis=MERGE_AXIS)

    Unet_output2 = UNet(input_2, norm_list)
# Unet_model2 = Model(inputs=[input_2, norm_list], outputs=Unet_output2)

    full_unet = Model(inputs=[img_input1,img_input2,img_input3,norm_list], outputs=Unet_output2)

# full_unet.summary()

    Style_output = Style_Encoder(256,img_free)
    Style_model = Model(inputs=img_free, outputs=Style_output)

    Mlp_output = MLP(Style_output,1536,2048)
    Mlp_model = Model(inputs=Style_output, outputs=Mlp_output)

    full_style = Model(inputs=img_free, outputs=Mlp_output)
    curr_epoch = 0
def scheduler(epoch):
    ep = 10
    if epoch < ep:
        return LEARNING_RATE
    else:
        return LEARNING_RATE * math.exp(0.1 * (ep - epoch))

Unet_optimizer = tf.keras.optimizers.Adam(LEARNING_RATE)
Style_optimizer = tf.keras.optimizers.Adam(0.00001)


try:
    with open(epoch_results_path, 'w', newline='') as epoch_file, open(step_results_path, 'w', newline='') as step_file:
        epoch_writer = csv.writer(epoch_file)
        step_writer = csv.writer(step_file)
        epoch_writer.writerow(['Epoch', 'Loss', 'L1_loss', 'SSIM_Score', 'PSNR', 'MSE', 'SSIM_Loss_val', 'L1_loss_val', 'SSIM_Score_val', 'PSNR_val', 'MSE_val'])
        step_writer.writerow(['Epoch', 'Step', 'Avg_Loss', 'Avg_L1_loss', 'Avg_SSIM_Score', 'Avg_PSNR'])

        for epoch in range(curr_epoch,NB_EPOCH):
            print(f'Starting epoch {epoch + 1}')
            ssim_scores, losses, l1_losses, pnsr_values, l2_losses = [], [], [], [], []
            
            new_learning_rate = scheduler(epoch)
            Unet_optimizer.learning_rate.assign(new_learning_rate)
            
            for step, ([Slice1, Slice2, Slice3], FreeImage) in enumerate(train_dataset):
                with tf.GradientTape() as tape:
                    norm_list_output = full_style(Slice2, training=True)
                    output1 = full_unet([Slice1, Slice2, Slice3, norm_list_output], training=True)
                    ssim_scores.append(ssim_score(FreeImage, output1))
                    loss_value = total_loss(FreeImage, output1)
                    pnsr_values.append(psnr(FreeImage, output1))
                    losses.append(loss_value)
                    l2_losses.append(l2_loss(FreeImage, output1))

                Unet_gradients = tape.gradient(loss_value, full_unet.trainable_variables)
                Unet_optimizer.apply_gradients(zip(Unet_gradients, full_unet.trainable_variables))

                with tf.GradientTape() as tape:
                    recon_norm_list = full_style(output1)
                    L1_loss_value = l1_loss(norm_list_output, recon_norm_list)
                    l1_losses.append(L1_loss_value)

                Style_gradients = tape.gradient(L1_loss_value, full_style.trainable_variables)
                Style_optimizer.apply_gradients(zip(Style_gradients, full_style.trainable_variables))

                if step % 100 == 0:
                    step_writer.writerow([epoch + 1, step, "{:.4f}".format(sum(losses)/len(losses)), sum(l1_losses)/len(l1_losses), "{:.4f}".format(sum(ssim_scores)/len(ssim_scores)), "{:.4f}".format(sum(pnsr_values)/len(pnsr_values))])
                    step_file.flush()  # Ensure data is written to file
#                     plt.imshow(output1[2],cmap='gray')
#                     plt.show()
                    print(f'Epoch {epoch + 1}, Step {step}, Avg_Loss: {"{:.4f}".format(sum(losses)/len(losses))}, Avg_L1_loss: {sum(l1_losses)/len(l1_losses)}, Avg_SSIM_Score: {"{:.4f}".format(sum(ssim_scores)/len(ssim_scores))}, Avg_PSNR: {"{:.4f}".format(sum(pnsr_values)/len(pnsr_values))}, Avg_L2_Loss: {"{:.4f}".format(sum(l2_losses)/len(l2_losses))}')

            print(f"================================Epoch {epoch+1} Validation ===========================================")
            ssim_scores_val, losses_val, l1_losses_val, pnsr_values_val, l2_losses_val = [], [], [], [], []

            for step, ([Slice1, Slice2, Slice3], FreeImage) in enumerate(validation_dataset):
                norm_list_val = full_style(Slice2, training=True)
                output1_val = full_unet([Slice1, Slice2, Slice3, norm_list_val], training=True)
                ssim_scores_val.append(ssim_score(FreeImage, output1_val))
                loss_value = total_loss(FreeImage, output1_val)
                pnsr_values_val.append(psnr(FreeImage, output1_val))
                losses_val.append(loss_value)
                l2_losses_val.append(l2_loss(FreeImage, output1_val))
                recon_norm_list_val = full_style(output1_val)
                L1_loss_value = l1_loss(norm_list_val, recon_norm_list_val)
                l1_losses_val.append(L1_loss_value)

            full_unet.save(f"/kaggle/working/full_unet_epoch_{epoch + 1}.h5")
            full_style.save(f"/kaggle/working/full_style_epoch_{epoch + 1}.h5")
            epoch_writer.writerow([epoch + 1, "{:.4f}".format(sum(losses)/len(losses)), float(sum(l1_losses)/len(l1_losses)), "{:.4f}".format(sum(ssim_scores)/len(ssim_scores)), "{:.4f}".format(sum(pnsr_values)/len(pnsr_values)), "{:.4f}".format(sum(l2_losses)/len(l2_losses)), "{:.4f}".format(sum(losses_val)/len(losses_val)), float(sum(l1_losses_val)/len(l1_losses_val)), "{:.4f}".format(sum(ssim_scores_val)/len(ssim_scores_val)), "{:.4f}".format(sum(pnsr_values_val)/len(pnsr_values_val)), "{:.4f}".format(sum(l2_losses_val)/len(l2_losses_val))])
            epoch_file.flush()  # Ensure data is written to file
            
            
except Exception as e:
    print(f"An error occurred: {e}")